In [1]:
using Mocha

Configuring Mocha...
 * CUDA       disabled by default
 * Native Ext disabled by default
Mocha configured, continue loading module...
DefaultBackend = Mocha.CPUBackend


In [2]:
train_filename = "cc_train.txt"
test_filename = "cc_test.txt"
exp_dir = "snapshots/cc_net/"

"snapshots/cc_net/"

In [3]:
data_layer  = HDF5DataLayer(name="train-data", source="cc_data/txt/$train_filename",
batch_size=64, shuffle=true)

Mocha.HDF5DataLayer(train-data)

In [4]:
fc1_layer  = InnerProductLayer(name="ip1", output_dim=256,
neuron=Neurons.ReLU(), bottoms=[:data], tops=[:ip1])

Mocha.InnerProductLayer(ip1)

In [5]:
fc2_layer  = InnerProductLayer(name="ip2", output_dim=2,
bottoms=[:ip1], tops=[:ip2])

Mocha.InnerProductLayer(ip2)

In [6]:
loss_layer = SoftmaxLossLayer(name="loss", bottoms=[:ip2,:label])

Mocha.SoftmaxLossLayer(loss)

In [7]:
backend = CPUBackend()
init(backend)

In [8]:
common_layers = [fc1_layer, fc2_layer]

2-element Array{Mocha.InnerProductLayer,1}:
 Mocha.InnerProductLayer(ip1)
 Mocha.InnerProductLayer(ip2)

In [9]:
net = Net("cc-train", backend, [data_layer, common_layers..., loss_layer])

[2018-10-14T22:05:48 | info | Mocha]: Constructing net cc-train on Mocha.CPUBackend...
[2018-10-14T22:05:49 | info | Mocha]: Topological sorting 4 layers...
[2018-10-14T22:05:49 | info | Mocha]: Setup layers...
[2018-10-14T22:05:51 | info | Mocha]: Network constructed!


************************************************************
          NAME: cc-train
       BACKEND: Mocha.CPUBackend
  ARCHITECTURE: 4 layers
............................................................
 *** Mocha.HDF5DataLayer(train-data)
    Outputs ---------------------------
          data: Blob(24 x 64)
         label: Blob(64)
............................................................
 *** Mocha.InnerProductLayer(ip1)
    Inputs ----------------------------
          data: Blob(24 x 64)
    Outputs ---------------------------
           ip1: Blob(256 x 64)
............................................................
 *** Mocha.InnerProductLayer(ip2)
    Inputs ----------------------------
           ip1: Blob(256 x 64)
    Outputs ---------------------------
           ip2: Blob(2 x 64)
............................................................
 *** Mocha.SoftmaxLossLayer(loss)
    Inputs ----------------------------
           ip2: Blob(2 x 64)
         label: Blob(64)
***

In [10]:
method = SGD()
params = make_solver_parameters(method, max_iter=10000, regu_coef=0.0005,
    mom_policy=MomPolicy.Fixed(0.9),
    lr_policy=LRPolicy.Inv(0.03, 0.0001, 0.75),
    load_from=exp_dir)

Dict{Symbol,Any} with 5 entries:
  :lr_policy  => Mocha.LRPolicy.Inv(0.03, 0.0001, 0.75)
  :max_iter   => 10000
  :load_from  => "snapshots/cc_net/"
  :regu_coef  => 0.0005
  :mom_policy => Mocha.MomPolicy.Fixed(0.9)

In [11]:
solver = Solver(method, params)

Mocha.Solver{Mocha.SGD}(Mocha.SGD(), Dict{Symbol,Any}(Pair{Symbol,Any}(:lr_policy, Mocha.LRPolicy.Inv(0.03, 0.0001, 0.75)),Pair{Symbol,Any}(:max_iter, 10000),Pair{Symbol,Any}(:load_from, "snapshots/cc_net/"),Pair{Symbol,Any}(:regu_coef, 0.0005),Pair{Symbol,Any}(:mom_policy, Mocha.MomPolicy.Fixed(0.9))), Mocha.CoffeeLounge("", 1, :merge, Dict{AbstractString,Dict{Int64,AbstractFloat}}(), Mocha.CoffeeBreak[], true, 4656669904, 4656669936))

In [12]:
setup_coffee_lounge(solver, save_into="$exp_dir/statistics.jld", every_n_iter=1000)

:merge

In [13]:
add_coffee_break(solver, TrainingSummary(), every_n_iter=100)
add_coffee_break(solver, Snapshot(exp_dir), every_n_iter=500)

2-element Array{Mocha.CoffeeBreak,1}:
 Mocha.CoffeeBreak(Mocha.TrainingSummary(Any[:iter, :obj_val]), 100, 0)
 Mocha.CoffeeBreak(Mocha.Snapshot("snapshots/cc_net/"), 500, 0)        

In [14]:
data_layer_test = HDF5DataLayer(name="test-data", source="cc_data/txt/$test_filename", batch_size=100)
acc_layer = AccuracyLayer(name="test-accuracy", bottoms=[:ip2, :label])
test_net = Net("cc-test", backend, [data_layer_test, common_layers..., acc_layer])

[2018-10-14T22:05:55 | info | Mocha]: Constructing net cc-test on Mocha.CPUBackend...
[2018-10-14T22:05:55 | info | Mocha]: Topological sorting 4 layers...
[2018-10-14T22:05:55 | info | Mocha]: Setup layers...
[2018-10-14T22:05:55 | info | Mocha]: Network constructed!


************************************************************
          NAME: cc-test
       BACKEND: Mocha.CPUBackend
  ARCHITECTURE: 4 layers
............................................................
 *** Mocha.HDF5DataLayer(test-data)
    Outputs ---------------------------
          data: Blob(24 x 100)
         label: Blob(100)
............................................................
 *** Mocha.InnerProductLayer(ip1)
    Inputs ----------------------------
          data: Blob(24 x 100)
    Outputs ---------------------------
           ip1: Blob(256 x 100)
............................................................
 *** Mocha.InnerProductLayer(ip2)
    Inputs ----------------------------
           ip1: Blob(256 x 100)
    Outputs ---------------------------
           ip2: Blob(2 x 100)
............................................................
 *** Mocha.AccuracyLayer(test-accuracy)
    Inputs ----------------------------
           ip2: Blob(2 x 100)
         label: B

In [15]:
add_coffee_break(solver, ValidationPerformance(test_net), every_n_iter=1000);

In [16]:
@time solve(solver, net)

destroy(net)
destroy(test_net)
shutdown(backend)

[2018-10-14T22:06:16 | info | Mocha]: Snapshot directory snapshots/cc_net/ already exists
[2018-10-14T22:06:16 | info | Mocha]:  TRAIN iter=000000 obj_val=35.97789383
[2018-10-14T22:06:17 | info | Mocha]: Saving snapshot to snapshot-000000.jld...
[2018-10-14T22:06:20 | info | Mocha]: 
[2018-10-14T22:06:20 | info | Mocha]: ## Performance on Validation Set after 0 iterations
[2018-10-14T22:06:20 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:06:20 | info | Mocha]:   Accuracy (avg over 6000) = 21.8667%
[2018-10-14T22:06:20 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:06:20 | info | Mocha]: 
[2018-10-14T22:06:22 | info | Mocha]:  TRAIN iter=000100 obj_val=0.52573615
[2018-10-14T22:06:22 | info | Mocha]:  TRAIN iter=000200 obj_val=0.63660085
[2018-10-14T22:06:22 | info | Mocha]:  TRAIN iter=000300 obj_val=0.42273435
[2018-10-14T22:06:22 | info | Mocha]:  TRAIN iter=000400 obj_val=0.46740746
[2018-10-14T2

[2018-10-14T22:06:27 | info | Mocha]:   Accuracy (avg over 6000) = 21.8667%
[2018-10-14T22:06:27 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:06:27 | info | Mocha]: 
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006100 obj_val=NaN
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006200 obj_val=NaN
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006300 obj_val=NaN
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006400 obj_val=NaN
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006500 obj_val=NaN
[2018-10-14T22:06:27 | info | Mocha]: Saving snapshot to snapshot-006500.jld...
[2018-10-14T22:06:27 | info | Mocha]:  TRAIN iter=006600 obj_val=NaN
[2018-10-14T22:06:28 | info | Mocha]:  TRAIN iter=006700 obj_val=NaN
[2018-10-14T22:06:28 | info | Mocha]:  TRAIN iter=006800 obj_val=NaN
[2018-10-14T22:06:28 | info | Mocha]:  TRAIN iter=006900 obj_val=NaN
[2018-10-14T22:06:28 | info | Mocha]:  TRAIN iter=007000 obj_val=NaN
[2018-10-14T22:06:2

Dict{AbstractString,Array{Mocha.AbstractParameter,1}} with 0 entries